# Encode/Decode SpectrumFrame payloads to/from base64 JSON

This notebook demonstrates how to encode and decode SpectrumFrame objects to/from base64 JSON format. SpectrumFrame are data structures that contain spectral information such as power spectral density (PSD), resolution bandwidth (RBW), start frequency and other parameters related to spectral analysis.

Through the following examples, we will see how to:
- Create a SpectrumFrame with example data
- Encode it to a base64 JSON string
- Decode it back to a SpectrumFrame object 
- Verify that the conversion was lossless


In [2]:
from rfscope.dsp.spectrum_codec import spectrumframe_to_b64, b64_to_spectrumframe
from rfscope.common.models import SpectrumFrame
import numpy as np

In this example we generate a SpectrumFrame object with synthetic test data, containing:

- A random power spectral density (PSD) signal generated using `np.random.normal` with 256 points
- Resolution bandwidth (RBW) of 1 Hz 
- Start frequency of 100 MHz
- 10 averages
- Noise floor of -150 dBm/Hz
- Metadata indicating this is a manual test

This simulated data will allow us to test the base64 encoding/decoding functions.


In [3]:
spectrumframe = SpectrumFrame(
    psd_dbm_per_hz=np.random.normal(size=2 ** 8),
    rbw_hz=1.0,
    f_start_hz=100e6,
    vbw_hz=None,
    window=None,
    averages=10,
    noise_floor_dbm_per_hz=-150.0,
    metadata={"source": "manual-test"},
)
spectrumframe

SpectrumFrame(psd_dbm_per_hz=array([-2.90390439e-01,  2.96514868e-01,  1.13214475e+00, -2.15951969e-01,
        6.10971187e-02,  7.56797451e-01, -6.41394293e-01, -1.07710329e+00,
        9.37852855e-02,  4.03087846e-01, -1.11191194e+00, -1.31474563e+00,
        1.38460563e+00,  2.80364919e-02, -7.97202226e-01, -4.69981883e-01,
       -5.09088038e-01,  1.49064769e-01,  7.39393714e-01, -4.56137410e-01,
        3.87922498e-01,  3.61958357e-01, -8.97956591e-04,  1.27519628e+00,
        8.10173189e-01,  8.16632943e-01, -1.77248055e+00,  7.35695072e-01,
        1.13963095e+00,  1.23228229e+00,  5.51797169e-01,  1.12878582e+00,
       -1.15695867e+00,  1.17087965e+00, -4.33505419e-01, -1.57985116e+00,
        5.63304106e-01,  8.06451968e-01,  2.92048992e+00,  1.19862530e+00,
        2.70530762e-01,  9.79187509e-01,  1.53646196e-01, -3.64049151e-01,
       -1.41724674e-01,  7.82312355e-01,  2.19420743e-01, -1.66833177e-01,
       -1.40591969e+00, -1.08242796e-01, -2.08244381e+00, -1.03901634e+

The `spectrumframe_to_b64` function converts a SpectrumFrame object into a base64-encoded JSON string. It:

1. Accepts a SpectrumFrame object as input, which contains spectral data like PSD values, RBW, start frequency, etc.
2. Takes optional parameters:
   - level: Compression level (1-9) for the zlib algorithm
   - dtype: NumPy data type for PSD values (e.g. np.float32)
3. Compresses the PSD data array using zlib
4. Creates a JSON object with the frame's metadata and compressed PSD
5. Encodes the JSON as base64
6. Returns the encoded string for efficient data transmission or storage

This encoding allows SpectrumFrame objects to be serialized and later reconstructed using `b64_to_spectrumframe`.


In [4]:
b64 = spectrumframe_to_b64(spectrumframe, level=6, dtype=np.float32)
b64

'{"header":{"rbw_hz":1.0,"f_start_hz":100000000.0,"vbw_hz":null,"window":null,"averages":10,"noise_floor_dbm_per_hz":-150.0,"metadata":{"source":"manual-test"},"psd_dtype":"float32","codec":"zlib+npy","version":1},"data":"eJydyPtX03UYwHEgxKSB0cn0SBwQ0BHCSeQ21/Z5voISnohrkOYlgQ3huKFszsWMrDMRIuI2LhtIyDXtyGKtpcL2eSYYN40ABQJEOBUIWgJeBnqysn+h92/vlyoy/t3oPdZWx61OsAVCabKEzXVj81IC2D5u7JQjkmOSxPQPj0gEwv88LFEkFT53aWriUeHz99oaGOTzho9bltv/zt5RW0qv91cQ17tFoHTfT58wCr7CtB0Ob/XE7Nuf46evmPm5iX1kQ1wBlu86h+1cHfRcmOZJ1kSgvHaBquNs8EjDOjIbzodf8uZoV3UnqYq9TEYFh1vrttWD8lAUjDvGQOPUr6hM4IF4shjEbdVQGuIAjh8Xwba1pZg0WQaiRxO0iNOL+oCXoV4aCVFzXKZApAEbZR558YIC0uNdSJOglYa5OdGaneEw9O1BEjPsTX0f6zFFO2FqD1thzvr7FCovt2DGtQljUlwA3uSfRRer79GQIIIfGiJNkvF8PN1pwO0ewcySpQ5+c3aHAzNc9N7rR9psPeGmqwpy74Qiq/oc9RrW4Z7EfKNitp7ka7po/k9XTXbrtkDVQBPqhy2kvU2Df7k0YoSlivzcYM9w5ovovcBUvH6lAGpOrcW3tEqQV3TwDiVnE/dwF8qqs8UIgT28PVdBzzf8Thc5XSaxz3lcf/Y6ht56AR8WRaIn14puaUlGq/SreOnQP8Tmu1rYE5wK9+/Xgc38m/AVa5ryY0swPmQl0zh3kfJC3DB+vxR2yFuxr0tDBzxeR/cRP5

The `b64_to_spectrumframe` function converts a base64-encoded JSON string back into a SpectrumFrame object. It performs these steps:

1. Takes a base64-encoded string as input that contains compressed spectral data and metadata
2. Decodes the base64 string back to a JSON object
3. Decompresses the PSD data array using zlib decompression
4. Reconstructs a complete SpectrumFrame object with:
   - Power Spectral Density (PSD) values
   - Resolution bandwidth (RBW)
   - Start frequency
   - Other parameters like VBW, window type, averages
   - Metadata

This function is the reverse operation of `spectrumframe_to_b64`, allowing spectral data to be reconstructed after transmission or storage.


In [5]:
spectrumframe_dec = b64_to_spectrumframe(b64)
spectrumframe_dec

SpectrumFrame(psd_dbm_per_hz=array([-2.90390432e-01,  2.96514869e-01,  1.13214481e+00, -2.15951964e-01,
        6.10971190e-02,  7.56797433e-01, -6.41394317e-01, -1.07710326e+00,
        9.37852859e-02,  4.03087854e-01, -1.11191189e+00, -1.31474566e+00,
        1.38460565e+00,  2.80364919e-02, -7.97202229e-01, -4.69981879e-01,
       -5.09088039e-01,  1.49064764e-01,  7.39393711e-01, -4.56137419e-01,
        3.87922496e-01,  3.61958355e-01, -8.97956605e-04,  1.27519631e+00,
        8.10173213e-01,  8.16632926e-01, -1.77248061e+00,  7.35695064e-01,
        1.13963091e+00,  1.23228228e+00,  5.51797152e-01,  1.12878585e+00,
       -1.15695870e+00,  1.17087960e+00, -4.33505416e-01, -1.57985115e+00,
        5.63304126e-01,  8.06451976e-01,  2.92049003e+00,  1.19862533e+00,
        2.70530760e-01,  9.79187489e-01,  1.53646201e-01, -3.64049137e-01,
       -1.41724676e-01,  7.82312334e-01,  2.19420746e-01, -1.66833177e-01,
       -1.40591967e+00, -1.08242795e-01, -2.08244371e+00, -1.03901637e+

The comparison `spectrumframe == spectrumframe_dec` verifies if the original SpectrumFrame object is equal to the decoded SpectrumFrame object after the base64 encoding and decoding process. This validates that the conversion was lossless and all the data (PSD values, RBW, start frequency, metadata, etc.) was preserved correctly during the encoding and decoding operations.


In [6]:
spectrumframe == spectrumframe_dec

True